<a href="https://colab.research.google.com/github/blue0620/aozorasummary/blob/main/plamo_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Plamo-2-1Bを利用した要約モデル学習のチュートリアル

有料版Google Colaboratoryのみで動作します。ランタイムのタイプをA100に設定してください。


## 環境構築
参考：https://qiita.com/autotaker1984/items/32109944a6a058161eee

In [ ]:
!pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu121
!pip install mamba_ssm==2.2.2 causal_conv1d==1.4.0
!pip install trl==0.12.2 bitsandbytes datasets

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.1/799.1 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 86.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#取得済みの要約文をダウンロードする
!wget https://github.com/blue0620/aozorasummary/raw/refs/heads/main/ccbync/summary.tsv

--2025-02-11 13:00:50--  https://github.com/blue0620/aozorasummary/raw/refs/heads/main/ccbync/summary.tsv
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/blue0620/aozorasummary/refs/heads/main/ccbync/summary.tsv [following]
--2025-02-11 13:00:50--  https://raw.githubusercontent.com/blue0620/aozorasummary/refs/heads/main/ccbync/summary.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7570878 (7.2M) [application/octet-stream]
Saving to: ‘summary.tsv’

summary.tsv         100%[===================>]   7.22M  --.-KB/s    in 0.03s   

2025-02-11 13:00:51 (229 MB/s) - ‘summary.tsv’ saved [757087

## データセットのロード

In [ ]:
from datasets import load_dataset, Dataset
import transformers
import pandas as pd

max_seq_length=6000

summary_df=pd.read_csv("summary.tsv",sep="\t",dtype=str)
summary_df=summary_df[["id","summary"]]
print(summary_df)
ds = load_dataset('globis-university/aozorabunko-clean')
df = pd.DataFrame(ds["train"])
df["id"]=[s["作品ID"]  for s in df["meta"]]
df=df[["id","text"]]
resdf=pd.merge(summary_df,df)
dataset=Dataset.from_pandas(resdf, split='train')

          id                                            summary
0     004407  この物語は、ラザルスという人物が死から蘇った後の出来事を語っています。ラザルスは死から蘇った...
1     053796  この文章は、文吉という人物が操という友人を訪ねようとするも、様々な葛藤や不安に駆られ、最終的...
2     053717  この文章は、シュールで抽象的なイメージや言葉の連なりで構成されています。各フレーズは独特な視...
3     053691  この文章は、独特なスタイルと抽象的な表現で書かれており、様々なイメージや概念を交差させている...
4     053692  この文章は、1931年6月18日に書かれた詩のようです。詩は、自然の描写、感情の表現、そして...
...      ...                                                ...
1939  059261  この物語は、悪魔が作った不思議な鏡から始まります。この鏡は、良いものや美しいものを小さく醜く...
1940  042387  この物語は、魔法使いの悪魔が作った不思議な鏡から始まります。この鏡は、美しいものや善良なもの...
1941  060151  昔、傲慢で悪い王様がいました。世界征服を夢見て、人々を恐怖に陥れようとしていました。王様は火...
1942  056574  この物語は、川音清兵衛という貧しい武士が、朝鮮征伐に出陣する際に、やせ細った馬を三両で購入す...
1943  004251  この物語は、名前のない一匹の犬の人生を描いています。犬は冬の間、飢えと孤独に苦しみ、人間や他...

[1944 rows x 2 columns]


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.96k [00:00<?, ?B/s]

aozorabunko-dedupe-clean.jsonl.gz:   0%|          | 0.00/241M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16951 [00:00<?, ? examples/s]

## peftを利用したLoRAによるモデルの学習

In [ ]:
import re
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM,TrainingArguments
from peft import LoraConfig, TaskType
from peft import get_peft_model
from trl import SFTTrainer

max_seq_length=6000
model_name = "pfnet/plamo-2-1b"

output_dir="outputs-plamo"
run_name="plamo-2-1b"

model = AutoModelForCausalLM.from_pretrained(
        model_name, torch_dtype=torch.bfloat16, device_map="cuda:0",trust_remote_code=True)

tokenizer = AutoTokenizer.from_pretrained(
    model_name, model_max_length=max_seq_length, add_eos_token=True,
    padding_side="right", use_fast=False,trust_remote_code=True)
tokenizer.pad_token = tokenizer.unk_token

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.01,
    bias="none",
    #target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",]
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

def formatting_prompts_func(examples):
    questions = examples["text"]
    responses = examples["summary"]
    texts = []
    for question, response in zip(questions, responses):
        text = f"### 本文: {question}\n ### 要約: {response}"+tokenizer.eos_token
        texts.append(text)
    return texts



trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    formatting_func=formatting_prompts_func,
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs=1,
        learning_rate = 2e-5,
        fp16 = False,
        bf16 = True,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs-plamo",
        report_to = "none", # Use this for WandB etc
    ),
)
trainer_stats = trainer.train()

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

modeling_plamo.py:   0%|          | 0.00/63.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/pfnet/plamo-2-1b:
- modeling_plamo.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/usr/local/lib/python3.11/dist-packages/mamba_ssm/ops/selective_scan_interface.py:163: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/mamba_ssm/ops/selective_scan_interface.py:239: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/usr/local/lib/python3.11/dist-packages/mamba_ssm/ops/triton/layer_norm.py:985: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/pyth

model.safetensors.index.json:   0%|          | 0.00/19.5k [00:00<?, ?B/s]

model-00001-of-00007.safetensors:   0%|          | 0.00/4.06G [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_num_proc. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:314: UserWarning: You passed a `dataset_num_proc` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
Parameter 'function'=<function SFTTrainer._prepare_non_packed_dataloader.<locals>.tokenize at 0x7cc3c01b62a0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't b

trainable params: 1,572,864 || all params: 1,293,014,784 || trainable%: 0.1216


Map (num_proc=2):   0%|          | 0/1944 [00:00<?, ? examples/s]

Step,Training Loss
1,3.890900
2,3.880300
3,4.114300
4,3.855300
5,4.018800
6,3.649000
7,3.874300
8,4.513800
9,3.828300
10,4.003900


In [ ]:
#!cp outputs-plamo ./drive/MyDrive/outputs-plamo -r